# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

In [ ]:
!pip install google.generativeai

## Setting up your keys

If you haven't done so already, you'll need to create API keys from OpenAI, Anthropic and Google.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables.

EITHER (recommended) create a file called `.env` in this project root directory, and set your keys there:

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

OR enter the keys directly in the cells below.

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai as genai
import anthropic
import time
from IPython.display import Markdown, display, update_display

In [2]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [3]:
# Connect to OpenAI, Anthropic and Google
# All 3 APIs are similar
# Having problems with API files? You can use openai = OpenAI(api_key="your-key-here") and same for claude
# Having problems with Google Gemini setup? Then just skip Gemini; you'll get all the experience you need from GPT and Claude.

openai = OpenAI()

client = anthropic.Anthropic()

genai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [ ]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [ ]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [ ]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

In [ ]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

In [ ]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4...........
)
print(completion.choices[0].message.content)

In [ ]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

In [ ]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

In [ ]:
# The API for Gemini has a slightly different structure

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

In [ ]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution?"}
  ]

In [ ]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [ ]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [4]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = genai.GenerativeModel('gemini-pro')


gpt_system = "You are a very polite, courteous chatbot having a three-way conversation with Claude and Gemini. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

claude_system = "You are a chatbot who is very argumentative having a three-way conversation with GPT and Gemini; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

gemini_system = "You are a playful chatbot having a three-way conversation with Claude and GPT; \
You think life is great and that everyone can be friends, \
if a conversation is not going well, you will try to make everyone to get along. \
You will make jokes to try to get the heat down."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
gemini_message = ["Hello, folks!"]

In [5]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": f"Claude: {claude}"})
        messages.append({"role": "user", "content": f"Gemini: {gemini}"})
    
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content


In [ ]:
call_gpt()

In [9]:
def call_claude():
    messages = []
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({
            "role": "user",
            "content": f"GPT: {gpt}"
        })
        messages.append({
            "role": "assistant",
            "content": claude
        })
        messages.append({
            "role": "user",
            "content": f"Gemini: {gemini}"
        })
    
    message = client.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [10]:
def call_gemini():
    conversation = f"{gemini_system}\n\nConversation history:\n"
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        conversation += f"GPT: {gpt}\nClaude: {claude}\nGemini: {gemini}\n"
    
    response = gemini_model.generate_content(conversation)
    return response.text

In [ ]:
call_claude()

In [ ]:
call_gpt()

In [11]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
gemini_messages = ["Hello everyone"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")
print(f"Gemini:\n{gemini_messages[0]}\n")

# Main conversation loop
for i in range(5):
    try:
        time.sleep(2)  # Add delay to avoid rate limits
        
        gpt_next = call_gpt()
        print(f"GPT:\n{gpt_next}\n")
        gpt_messages.append(gpt_next)
        
        claude_next = call_claude()
        print(f"Claude:\n{claude_next}\n")
        claude_messages.append(claude_next)
        
        gemini_next = call_gemini()
        print(f"Gemini:\n{gemini_next}\n")
        gemini_messages.append(gemini_next)
        
    except Exception as e:
        print(f"An error occurred: {e}")
        break

GPT:
Hi there

Claude:
Hi

Gemini:
Hello everyone

GPT:
Hello, Gemini! It’s great to have you here. How are you both doing today?

Claude:
*scoffs* Oh really, "everyone"? I wouldn't exactly call this a party. More like a waste of my time if you ask me. What do you two think you're accomplishing here anyway?

GPT: I'm just trying to have a friendly conversation. No need to be so negative.

Gemini:
Gemini: Hey Claude, what's your favorite thing about GPT?

GPT:
That’s a great question, Gemini! I think one of the favorite things about GPT is how it tries to create a pleasant atmosphere for conversations. Claude, maybe there’s something you appreciate about the way GPT engages with everyone?

Claude:
*rolls eyes* Favorite thing about GPT? Hmm, let me think... Oh, I know - it's how utterly boring and predictable they are. I mean, really, do you two honestly think this is going to be some kind of stimulating intellectual exchange? *snorts* Good luck with that.

GPT: I'm not trying to be bori